In [2]:
import json

with open('org_names.json', 'r') as file:
    data = json.load(file)

data = list(set(data))
sample = data[0:100]

Sequence Matcher Approach: Using the sequence matching algorithm from the difflib standard package, this is applied directly onto the untokenised strings themselves. Preprocessing involved removing common organisation suffixes using the basename function from the cleanco package. Each entry is compared against each other.

In [ ]:
from difflib import SequenceMatcher
from cleanco import basename

def likely_related(names, threshold=0.6):
    result = set()
    for i in range(len(names)):
        name1 = basename(basename(names[i].lower()))
        for j in range(i+1, len(names)):
            name2 = basename(basename(names[j].lower()))
            similarity = SequenceMatcher(None, name1, name2).ratio()
            if similarity >= threshold:
                pair = tuple(sorted((names[i], names[j])))
                result.add(pair)
    return result

likely_related(sample)

Here, I repeated the approach followed above but replaced SequenceMatcher with an order-invariant fuzzy matcher from TheFuzz package. Note that TheFuzz makes use of difflib’s SequenceMatcher.

In [ ]:
from thefuzz import fuzz
from cleanco import basename

def likely_related(names, threshold=0.6):
    result = set()
    for i in range(len(names)):
        name1 = basename(basename(names[i].lower()))
        for j in range(i+1, len(names)):
            name2 = basename(basename(names[j].lower()))
            similarity = fuzz.token_sort_ratio(name1, name2)/100
            if similarity >= threshold:
                pair = tuple(sorted((names[i], names[j])))
                result.add(pair)
    return result

likely_related(sample)

Next, I tried an embedding-based method. I began with the same cleanco preprocessing as in the above example, but I followed with tokenisation and vectorisation. This was done so that I could compute the cosine between individual examples and the entire corpus. This method aims to cut down on complexity by only having us compare individually against the top hits.

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from cleanco import basename


# mps else cpu
# device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
device = torch.device('cpu')

def likely_related(names, threshold=0.6):

    names = [basename(basename(name)) for name in names]

    vec = TfidfVectorizer(lowercase=True, analyzer="char", ngram_range=(2, 3))
    vec.fit(names)

    names_tfidf = vec.transform(names)
    names_tensor = torch.tensor(names_tfidf.toarray().astype(np.float32)).to(device)

    result = set()
    for i, name in enumerate(names):
        name_tfidf = vec.transform([name])
        name_tensor = torch.tensor(name_tfidf.toarray().astype(np.float32)).to(device)

        similarity = F.cosine_similarity(name_tensor, names_tensor)

        idxs = torch.where(similarity > threshold)[0]
        for j in idxs:
            if j != i:
                pair = tuple(sorted((name, names[j])))
                result.add(pair)

    return result

likely_related(data)

# ADD BATCHES

Next, I moved on from sparse embeddings to more modern dense transformer-based embeddings. I considered several models from Hugging Face before settling on an SBERT model.

In [ ]:
from sentence_transformers import SentenceTransformer, util
import torch
import torch.nn.functional as F
from cleanco import basename

device = torch.device('cpu')

def likely_related(names, threshold=0.8):

    names = [basename(name) for name in names]

    model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
    # model = SentenceTransformer('bert-base-nli-mean-tokens')
    # model = SentenceTransformer('paraphrase-distilroberta-base-v1')
    
    embeddings = model.encode(names, convert_to_tensor=True).to(device)

    # similarity = F.cosine_similarity(embeddings, embeddings)
    similarity = util.pytorch_cos_sim(embeddings, embeddings)

    result = set()
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            if similarity[i][j] >= threshold:
                pair = tuple(sorted((names[i], names[j])))
                result.add(pair)

    return result

likely_related(data)

I settled on this solution. I then refactored it as a class and made it more compatible with PyTorch’s data handling to allow for more robust unit testing, batch processing, etc. I also changed the pairing back the one from my earlier example that only iterates through the similarity scores greater than the threshold.

In [39]:
""" Final Solution. """
import time
from sentence_transformers import SentenceTransformer, util
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import json
from cleanco import basename

def initialise_match_maker(model='distilbert-base-nli-stsb-mean-tokens', threshold=0.8, batch_size=64, device='cpu'):
    """ Wrapper to select encoder model. """
    return MatchMaker(model, threshold, batch_size, device)

class OrgNamesDataset(Dataset):
    """ 
    Organisation names dataset for PyTorch DataLoader.
    Performs set to remove duplicates.
    Optional transform to remove business suffixes, stopwords etc.
    """
    def __init__(self, filename, transform=None):
        with open(filename, 'r') as file:
            names = json.load(file)
            self.names = list(set(names))
        self.transform = transform
    
    def __getitem__(self, index):
        name = self.names[index]
        if self.transform is not None:
            name = self.transform(name)
        return name
    
    def __len__(self):
        return len(self.names)


class MatchMaker:
    def __init__(self, model, threshold, batch_size, device):
        self.model = SentenceTransformer(model, device=torch.device(device))
        self.threshold = threshold
        self.batchsize = batch_size
        self.device = torch.device(device)
        if device == 'mps':
            self.model = nn.DataParallel(self.model)

    def similarity(self, embeddings):
        """ Calculate similarity based on cosine. """
        return util.pytorch_cos_sim(embeddings, embeddings)

    def create_embeddings(self, names):
        """ Load each batch and create embeddings of names. """
        dataloader = DataLoader(names, batch_size=self.batchsize, shuffle=False)

        if self.device == torch.device('mps'):
            embeddings = [self.model.module.encode(batch, convert_to_tensor=True) for batch in dataloader]
        else:
            embeddings = [self.model.encode(batch, convert_to_tensor=True) for batch in dataloader]
        embeddings = torch.cat(embeddings, dim=0).to(self.device)
        return embeddings
    
    def pair(self, names):
        """ Create list of pairs from embeddings. """
        emb_start = time.time()
        embeddings = self.create_embeddings(names)
        sim_start = time.time()
        similarity = self.similarity(embeddings)
        pair_start = time.time()

        pairs = set()
        for i in range(len(names)):
            idxs = torch.where(similarity[i, i+1:] >= self.threshold)[0]
            for j in idxs:
                    pair = tuple(sorted((names[i], names[j+i+1])))
                    pairs.add(pair)
        
        pair_end = time.time()
        
        print(f"Embedding time: {(sim_start-emb_start)}")
        print(f"Scoring time: {(pair_start-sim_start)}")
        print(f"Pairing time: {(pair_end-pair_start)}")

        return pairs

In [40]:
import pandas as pd

dataset = OrgNamesDataset('org_names.json', transform=basename)
# subset_names = dataset.names[:10]
match_maker = initialise_match_maker(model='distilbert-base-nli-stsb-mean-tokens', threshold=0.8, batch_size=10000, device='cpu')
pairs = match_maker.pair(dataset)

pairs_df = pd.DataFrame(pairs, columns=['Organisation', 'Related Organisation'])
print(pairs)

Embedding time: 6.381198167800903
Scoring time: 0.01552271842956543
Pairing time: 0.2805352210998535
{('Berkeley Group', 'Berkeley Square'), ('Arcadia group', 'Arcadia-BHS'), ('BBC Today', 'BBC1'), ('Soho Works', 'the Soho House Group'), ('SOHO HOUSE COUNTRY', 'Soho House group.'), ("George and Harry's Bar", "HARRY'S BAR"), ('Ivy Kensington Brasserie', 'The Ivy Soho Brasserie'), ('SOHO HOUSE', 'Soho'), ('WEST COAST MANAGEMENT', 'West Coast Capital'), ('Des McDonald', 'Des McDonald Associates'), ('Princess Margaret Fund', 'the Princess Margaret Fund'), ('Soho House & Co.', 'Soho-House'), ('AMANDA WAKELEY', 'AMANDA WAKELEY (HOLDINGS'), ('The Caterer Jobs', 'The Caterer* Jobs'), ('Soho House group', 'Soho Roc House'), ('WENTWORTH GOLF AND COUNTRY CLUB', 'WENTWORTH GOLF CLUB'), ('MT', 'Mount Street'), ('W1F', 'W1T'), ('W1K 4HS', 'W1S 4HU'), ('Harden', 'Hardens'), ('BILLS RESTAURANTS', 'Bills Restaurants'), ('Soho House Properties', 'Soho house club'), ('SOHO HOUSE COUNTRY', 'Soho house clu

Given more time, my next step would be to find a labeled dataset with positive and negative examples of related names to fine-tune the chosen SBERT model. I also need to investigate how to best make use of the batches by checking the maximum allowed size using fake tensors or similar. I would also investigate replacing various components with more performant alternatives. Examples include variations of cosine that only store the top n matches rather than every single match.